In [1]:
import pandas as pd

In [2]:
# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635
...,...,...,...,...,...,...,...,...,...,...,...
39165,39165,Donna Howard,F,12th,Thomas High School,99,90,14,Charter,1635,1043130
39166,39166,Dawn Bell,F,10th,Thomas High School,95,70,14,Charter,1635,1043130
39167,39167,Rebecca Tanner,F,9th,Thomas High School,73,84,14,Charter,1635,1043130
39168,39168,Desiree Kidd,F,10th,Thomas High School,99,90,14,Charter,1635,1043130


In [3]:
# Calculate these datas
totalNumberOfSchools = school_data["school_name"].count()
totalNumberOfStudents = student_data["student_name"].count()
totalBudget = "${:0,.2f}".format(school_data["budget"].sum())
averageMathScore = student_data["math_score"].mean()
averageReadingScore = student_data["reading_score"].mean()
totalNumberOfStudents

39170

In [4]:
# Calculate the percentage of Students passing math scores
students_passing_math = student_data[student_data.math_score >= 70]
student_passing_math_count = students_passing_math["student_name"].count()
student_passing_math_count

29370

In [5]:
# Calculate Students passing math
percentPassingMath = student_passing_math_count/totalNumberOfStudents * 100
percentPassingMath

74.9808526933878

In [6]:
# Calculate Students passing reading 70 or greater
students_passing_reading = student_data[student_data.reading_score >= 70]
student_passing_reading_count = students_passing_reading["student_name"].count()
student_passing_reading_count

33610

In [7]:
# Calculate Students percent on reading 70 or greater
percentPassingReading = student_passing_reading_count/totalNumberOfStudents * 100
percentPassingReading

85.80546336482001

In [8]:
# Calculate Students who passed math and reading or overall reading
over_all_passing_df = student_data.where((student_data.math_score >= 70) & (student_data.reading_score >= 70))
over_all_passing_count = over_all_passing_df["student_name"].count()
over_all_passing_percent = over_all_passing_count/totalNumberOfStudents * 100
over_all_passing_percent

65.17232575950983

In [9]:
# Place all of the data found into a summary DataFrame
summary_df = pd.DataFrame({"Total School": [totalNumberOfSchools],
                          "Total Students": [totalNumberOfStudents],
                          "Total Budget": [totalBudget],
                          "Average Math Score": [averageMathScore],
                          "Average Reading Score": [averageReadingScore],
                          "% Passing Math": [percentPassingMath],
                          "% Passing Reading": [percentPassingReading],
                          "% Overall Passing": [over_all_passing_percent]})
summary_df.style.format({'Total School':'{:,}','Total Students':'{:,}'})

,Total School,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.985371,81.877840,74.980853,85.805463,65.172326


In [10]:
# groups school_names by school_name name
school_name_group_df = school_data_complete.set_index('school_name').groupby(['school_name'])

# Get school types
sch_types = school_data.set_index('school_name')['type']

# Get total students by school
stu_per_sch = school_name_group_df['Student ID'].count()

# Get School budget
sch_budget = school_data.set_index('school_name')['budget']

# Get per student budget
stu_budget = school_data.set_index('school_name')['budget']/school_data.set_index('school_name')['size']

#avg scores by school_name
avg_math = school_name_group_df['math_score'].mean()
avg_read = school_name_group_df['reading_score'].mean()

# Calculate the % passing scores
pass_math = school_data_complete[school_data_complete['math_score'] >= 70].groupby('school_name')['Student ID'].count()/stu_per_sch 
pass_read = school_data_complete[school_data_complete['reading_score'] >= 70].groupby('school_name')['Student ID'].count()/stu_per_sch 
overall = school_data_complete[(school_data_complete['reading_score'] >= 70) & (school_data_complete['math_score'] >= 70)].groupby('school_name')['Student ID'].count()/stu_per_sch 

# Set up the summary dataframe
school_summary_df = pd.DataFrame({
    "School Type": sch_types,
    "Total Students": stu_per_sch,
    "Total School Budget": sch_budget,
    "Per Student Budget": stu_budget,
    "Average Math Score": avg_math,
    "Average Reading Score": avg_read,
    '% Passing Math': pass_math * 100,
    '% Passing Reading': pass_read * 100,
    "Overall Passing Rate": overall * 100
})

# Format the output
school_summary_df.style.format({'Total Students': '{:}', 
                          "Total School Budget": "${:,.2f}", 
                          "Per Student Budget": "${:.2f}"})

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
school_name,,,,,,,,,
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,89.227166
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


In [11]:
# Generate summary of top performing schools sort values by passing rate and print top 5 
top_5_schools = school_summary_df.sort_values("Overall Passing Rate", ascending = False)
print("Top 5 Schools by Overall Passing Rate")
top_5_schools.head().style.format({'Total Students': '{:}',
                           "Total School Budget": "${:,.2f}", 
                           "Per Student Budget": "${:.2f}"})

Top 5 Schools by Overall Passing Rate


,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
school_name,,,,,,,,,
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,90.948012
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,90.582567
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


In [12]:
# Generate summary of bottom performing schools sort values by passing rate and print bottom 5 
bottom_5_schools = top_5_schools.tail()
bottom_5_schools = bottom_5_schools.sort_values("Overall Passing Rate")
print("Bottom 5 Schools by Overall Passing Rate")
bottom_5_schools.head().style.format({'Total Students': '{:}',
                           "Total School Budget": "${:,.2f}", 
                           "Per Student Budget": "${:.2f}"})

Bottom 5 Schools by Overall Passing Rate


,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
school_name,,,,,,,,,
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,52.988247
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


In [13]:
# Setup grade levels and average math scores schools
ninth_grade_math = student_data.loc[student_data['grade'] == '9th'].groupby('school_name')["math_score"].mean()
tenth_grade_math = student_data.loc[student_data['grade'] == '10th'].groupby('school_name')["math_score"].mean()
eleventh_grade_math = student_data.loc[student_data['grade'] == '11th'].groupby('school_name')["math_score"].mean()
twelfth_grade_math = student_data.loc[student_data['grade'] == '12th'].groupby('school_name')["math_score"].mean()

math_scores_summary = pd.DataFrame({
        "9th": ninth_grade_math,
        "10th": tenth_grade_math,
        "11th": eleventh_grade_math,
        "12th": twelfth_grade_math
})
math_scores_summary = math_scores_summary[['9th', '10th', '11th', '12th']]
math_scores_summary.index.name = "School"
print("Math Scores by Grade")
math_scores_summary

Math Scores by Grade


,9th,10th,11th,12th
School,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


In [14]:
# Setup grade levels and average reading scores for schools
ninth_grade_reading = student_data.loc[student_data['grade'] == '9th'].groupby('school_name')["reading_score"].mean()
tenth_grade_reading = student_data.loc[student_data['grade'] == '10th'].groupby('school_name')["reading_score"].mean()
eleventh_grade_reading = student_data.loc[student_data['grade'] == '11th'].groupby('school_name')["reading_score"].mean()
twelfth_grade_reading = student_data.loc[student_data['grade'] == '12th'].groupby('school_name')["reading_score"].mean()

reading_scores_summary = pd.DataFrame({
        "9th": ninth_grade_reading,
        "10th": tenth_grade_reading,
        "11th": eleventh_grade_reading,
        "12th": twelfth_grade_reading
})
reading_scores_summary = reading_scores_summary[['9th', '10th', '11th', '12th']]
reading_scores_summary.index.name = "School"
print("Reading Scores by Grade")
reading_scores_summary


Reading Scores by Grade


,9th,10th,11th,12th
School,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


In [15]:
# Setup spending ranges
spending_ranges = [0, 584, 629, 644, 680]
group_name = ['< $585', "$585 - 630", "$630 - 645", "$645 - 680"]
school_data_complete['spending_ranges'] = pd.cut(school_data_complete['budget']/school_data_complete['size'], 
                                               spending_ranges, labels = group_name)

# Group by spending range
by_spending = school_data_complete.groupby('spending_ranges')

# Perform calculation for math and reading scores
avg_math = by_spending['math_score'].mean()
avg_read = by_spending['reading_score'].mean()
pass_math = school_data_complete[school_data_complete['math_score'] >= 70].groupby('spending_ranges')['Student ID'].count()/by_spending['Student ID'].count()
pass_read = school_data_complete[school_data_complete['reading_score'] >= 70].groupby('spending_ranges')['Student ID'].count()/by_spending['Student ID'].count()
overall = school_data_complete[(school_data_complete['reading_score'] >= 70) & (school_data_complete['math_score'] >= 70)].groupby('spending_ranges')['Student ID'].count()/by_spending['Student ID'].count()

            
# Build dataframe for scores by spending
scores_by_spend = pd.DataFrame({
    "Average Math Score": avg_math,
    "Average Reading Score": avg_read,
    '% Passing Math': pass_math * 100,
    '% Passing Reading': pass_read * 100,
    "% Overall Passing Rate": overall * 100
            
})
            
# Setup order of columns for display
scores_by_spend = scores_by_spend[[
    "Average Math Score",
    "Average Reading Score",
    '% Passing Math',
    '% Passing Reading',
    "% Overall Passing Rate"
]]

scores_by_spend.index.name = "Spending Ranges(Per Student)"
scores_by_spend = scores_by_spend.reindex(group_name)
print("Scores by School Spending")
scores_by_spend


Scores by School Spending


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Spending Ranges(Per Student),,,,,
< $585,83.363065,83.964039,93.702889,96.686558,90.640704
$585 - 630,79.982873,82.312643,79.109851,88.513145,70.939239
$630 - 645,77.821056,81.301007,70.623565,82.600247,58.841194
$645 - 680,77.049297,81.005604,66.230813,81.109397,53.528791


In [16]:
# Setup school size ranges
school_size_ranges = [0, 999, 1999, 5000]
group_name = ["Small (<1000)", "Medium (1000-2000)" , "Large (2000-5000)"]
school_data_complete['size_ranges'] = pd.cut(school_data_complete['size'], school_size_ranges, labels = group_name)

# Group by school size ranges
by_size = school_data_complete.groupby('size_ranges')

# Perform calculation for math and reading scores 
avg_math = by_size['math_score'].mean()
avg_read = by_size['math_score'].mean()
pass_math = school_data_complete[school_data_complete['math_score'] >= 70].groupby('size_ranges')['Student ID'].count()/by_size['Student ID'].count()
pass_read = school_data_complete[school_data_complete['reading_score'] >= 70].groupby('size_ranges')['Student ID'].count()/by_size['Student ID'].count()
overall = school_data_complete[(school_data_complete['reading_score'] >= 70) & (school_data_complete['math_score'] >= 70)].groupby('size_ranges')['Student ID'].count()/by_size['Student ID'].count()

            
# Build dataframe of scores by school size            
scores_by_size = pd.DataFrame({
    "Average Math Score": avg_math,
    "Average Reading Score": avg_read,
    '% Passing Math': pass_math * 100,
    '% Passing Reading': pass_read * 100,
    "% Overall Passing Rate": overall * 100
            
})
            
# Order columns for display
scores_by_size = scores_by_size[[
    "Average Math Score",
    "Average Reading Score",
    '% Passing Math',
    '% Passing Reading',
    "% Overall Passing Rate"
]]

scores_by_size.index.name = "Total Students"
scores_by_size = scores_by_size.reindex(group_name)
print("Scores by School Size")
scores_by_size

Scores by School Size


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Total Students,,,,,
Small (<1000),83.828654,83.828654,93.952484,96.040317,90.136789
Medium (1000-2000),83.372682,83.372682,93.616522,96.773058,90.624267
Large (2000-5000),77.477597,77.477597,68.652380,82.125158,56.574046


In [17]:
# Group by type of school
school_types = school_data_complete.groupby("type")

# Perform calculations for scores 
avg_math = school_types['math_score'].mean()
avg_read = school_types['math_score'].mean()
pass_math = school_data_complete[school_data_complete['math_score'] >= 70].groupby('type')['Student ID'].count()/school_types['Student ID'].count()
pass_read = school_data_complete[school_data_complete['reading_score'] >= 70].groupby('type')['Student ID'].count()/school_types['Student ID'].count()
overall = school_data_complete[(school_data_complete['reading_score'] >= 70) & (school_data_complete['math_score'] >= 70)].groupby('type')['Student ID'].count()/school_types['Student ID'].count()

# Build dataframe for scores by school type
scores_school_types = pd.DataFrame({
    "Average Math Score": avg_math,
    "Average Reading Score": avg_read,
    '% Passing Math': pass_math * 100,
    '% Passing Reading': pass_read * 100,
    "% Overall Passing Rate": overall * 100})
    
# Order columns for display
scores_school_types = scores_school_types[[
    "Average Math Score",
    "Average Reading Score",
    '% Passing Math',
    '% Passing Reading',
    "% Overall Passing Rate"
]]
scores_school_types.index.name = "Type of School"
print("Scores by School Type")
scores_school_types

Scores by School Type


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Type of School,,,,,
Charter,83.406183,83.406183,93.701821,96.645891,90.560932
District,76.987026,76.987026,66.518387,80.905249,53.695878
